NESNE TAKİP UYGULAMALARI

In [ ]:
#paketleri(kütüphaneleri) içe aktarıyoruz
import numpy as np 
import cv2

corner_track_params = dict(maxCorners = 10,qualityLevel =0.3,minDistance =7,blockSize=7) #gerekli parametrelerimizi burda tanımladık
lk_params = dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

cap = cv2.VideoCapture(0)  #görüntümüzü webcamden alıyoruz
ret,prev_frame= cap.read()  #ilk framemimizi okuyoruz
prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY) #ilk framemizi gri tona çeviriyoruz

prevPoints = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params) #ilk framedeki köşeleri buluruz 
mask = np.zeros_like(prev_frame) #bu görüntünün boyutunun eşleniği ama her elemanı sıfır olacak şeklinde bir maske oluşturuyoruz

while True:  #sonsuz bir görüntü başlatıyoruz
    ret,frame =cap.read() #sonraki framemizi okuyoruz
    frame =cv2.flip(frame,1)  #framelerimizi y eksenine göre döndürüyoruz
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  #sonraki framemizi gri tona çeviriyoruz

    nextPts,status,err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPoints,None,**lk_params)  # bir karedeki noktaların diğer karedeki karşılıklarını buluruz
    good_new = nextPts[status==1] 
    good_prev =prevPoints[status==1]

    for i,(new,prev) in enumerate(zip(good_new,good_prev)): # her takip edilebilen nokta için bir çizgi ve noktaların etrafına ise de bir daire çizeriz.(mask ve frame üzerine)
       
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()

        mask = cv2.line(mask,(int(x_new),int(y_new)),(int(x_prev),int(y_prev)),(0,255,0),3)

        frame =cv2.circle(frame,(int(x_new),int(y_new)),8,(0,0,255),-1)
    img = cv2.add(frame,mask)

    cv2.imshow("frame",img) #framelerimizi gösteririrz

    if cv2.waitKey(30) ==ord('q'): #q ' ya basılırsa döngüyü kır
        break
            
    prev_gray = frame_gray.copy()  #frame_gray'i kopyalayıp prev_gray değişkenine atarız
    prevPoints = good_new.reshape(-1,1,2)

cv2.destroyAllWindows() #tüm pencereleri kapatıp
cap.release() #video akışını serbest bırakırız


In [ ]:
#paketleri(kütüphaneleri) içe aktarıyoruz
import numpy as np 
import cv2


cap = cv2.VideoCapture(0)  #görüntümüzü webcamden alıyoruz
ret,frame1= cap.read()  #ilk framemimizi okuyoruz
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY) #ilk framemizi gri tona çeviriyoruz

hsv_mask = np.zeros_like(frame1) #bu frame'in boyutunda ama her elemanı sıfır olacak şeklinde bir maske oluşturuyoruz
hsv_mask[:,:,1] =255 #bu maskenin 1. kanalındaki değerleri 255 değerine eşitledik

while True: #ikinci frame için döngü oluşturduk
    ret,frame2 = cap.read()  #frameleri okuduk
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)  #okuduğumuz frameleri gri formata çevirdik

    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0)  # bir karedeki hareketin diğer karedeki karşılığını hesaplıyoruz 
    mag,ang =cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)   #HSV renk uzayından BGR renk uzayına dönüşüm yapıyoruz ve oluşturulan maskeyi renklendirip ve görüntülüyoruz
    cv2.imshow("frame2",bgr)
    if cv2.waitKey(30) ==ord('q'): #'q' tuşuna basılırsa döngüyü kır
        break

    prvsImg=nextImg #loop oluşturarak bir önceki frame'i  bir sonraki frame'e eşitleyerek ilerliyoruz
cv2.destroyAllWindows()  #tüm pencereleri kapatıp 
cap.release()  #video akışını serbest bırakıtık
